In [1]:
# --------------------- Imports ---------------------
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim
from google.colab import drive

# --------------------- Matplotlib Setup ---------------------
mpl.rcParams.update({
    'font.size': 14,
    'axes.titlesize': 15,
    'axes.labelsize': 12,
    'xtick.labelsize': 11,
    'ytick.labelsize': 11,
    'legend.fontsize': 11,
    'figure.dpi': 300,
    'savefig.dpi': 300,
    'figure.autolayout': True,
})

# --------------------- Load Data ---------------------
print("Mounting Google Drive and loading dataset...")
drive.mount('/content/drive')
total_capture_7k = pd.read_csv('drive/My Drive/correlation_wide.csv')
print(f"Loaded dataset with shape: {total_capture_7k.shape}")

# --------------------- Identify Unique Static Parameter Sets ---------------------
static_cols = [
    'MikeSorghum', 'Quartz', 'Plagioclase', 'Apatite', 'Ilmenite',
    'Diopside_Mn', 'Diopside', 'Olivine', 'Alkali-feldspar',
    'Montmorillonite', 'Glass', 'temp', 'shift', 'year'
]

# Add timestep count per file_id
file_lengths = total_capture_7k.groupby('file_id').size().rename("num_timesteps").reset_index()
static_rows = total_capture_7k.groupby('file_id')[static_cols].first().reset_index()
static_rows = static_rows.merge(file_lengths, on='file_id')

# Filter only unique static parameter sets
unique_static_rows = static_rows.drop_duplicates(subset=static_cols)
unique_file_ids = unique_static_rows['file_id'].tolist()

# --------------------- Extract Time Series Data ---------------------
filtered_df = total_capture_7k[total_capture_7k['file_id'].isin(unique_file_ids)].copy()

# Truncate each group to 101 timesteps
filtered_df = filtered_df.groupby('file_id').head(101).reset_index(drop=True)

# --------------------- Static Feature Table ---------------------
Input_Link_Table = filtered_df.groupby('file_id').agg({col: 'first' for col in static_cols}).reset_index()
print(f"Static feature table created: Input_Link_Table.shape = {Input_Link_Table.shape}")

# --------------------- Time Series Structuring ---------------------
result = filtered_df[['Total_CO2_capture', 'year', 'file_id']]
file_ids = result['file_id'].unique()
num_file_ids = len(file_ids)
max_timesteps = 101
relevant_data = np.zeros((num_file_ids, max_timesteps))
file_id_order = np.zeros(num_file_ids)

for i, file_id in enumerate(file_ids):
    file_data = result[result['file_id'] == file_id]['Total_CO2_capture'].values
    relevant_data[i, :len(file_data)] = file_data
    file_id_order[i] = file_id
print(f"Time series matrix constructed: relevant_data.shape = {relevant_data.shape}")

# --------------------- Clustering ---------------------
scaler = StandardScaler()
normalized_data = scaler.fit_transform(relevant_data)
kmeans = KMeans(n_clusters=8, random_state=42)
clusters = kmeans.fit_predict(normalized_data)
print("Performed KMeans clustering into 8 clusters")

# Compute boundary stats
cluster_boundaries = []
for cluster_id in range(8):
    cluster_data = normalized_data[clusters == cluster_id]
    min_v = scaler.inverse_transform(np.min(cluster_data, axis=0).reshape(1, -1)).flatten()
    median_v = scaler.inverse_transform(np.median(cluster_data, axis=0).reshape(1, -1)).flatten()
    mean_v = scaler.inverse_transform(np.mean(cluster_data, axis=0).reshape(1, -1)).flatten()
    max_v = scaler.inverse_transform(np.max(cluster_data, axis=0).reshape(1, -1)).flatten()
    cluster_boundaries.append((min_v, median_v, mean_v, max_v))
cluster_boundaries = np.array(cluster_boundaries)
print(f"Cluster boundary stats calculated: cluster_boundaries.shape = {cluster_boundaries.shape}")

# --------------------- Merge Static Features with Clusters ---------------------
Clustering_link_table = pd.DataFrame({'file_id': file_id_order.astype(int), 'cluster': clusters})
Clustering_link_table = Clustering_link_table.sort_values(by='file_id').reset_index(drop=True)
merged_df = pd.merge(Input_Link_Table, Clustering_link_table, on='file_id')
print(f"Final input features (static + cluster): merged_df.shape = {merged_df.shape}")

# --------------------- Create Output Time Series DataFrame ---------------------
data = [[file_id_order[i].astype(int), t, relevant_data[i, t]] for i in range(len(file_id_order)) for t in range(max_timesteps)]
df_output = pd.DataFrame(data, columns=['file_id', 'timestep', 'CO2']).sort_values(by=['file_id', 'timestep'])
print(f"Final output time series: df_output.shape = {df_output.shape}")

# --------------------- Summary ---------------------
print("Data Preparation Summary:")
print(f"Static Input Table: merged_df [{merged_df.shape[0]} rows × {merged_df.shape[1]} columns]")
print(f"Time Series Output: df_output [{df_output.shape[0]} rows × 3 columns]")
print(f"Cluster Boundaries: cluster_boundaries [{cluster_boundaries.shape}]")

Mounting Google Drive and loading dataset...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loaded dataset with shape: (1192157, 17)
Static feature table created: Input_Link_Table.shape = (2703, 15)
Time series matrix constructed: relevant_data.shape = (2703, 101)
Performed KMeans clustering into 8 clusters
Cluster boundary stats calculated: cluster_boundaries.shape = (8, 4, 101)
Final input features (static + cluster): merged_df.shape = (2703, 16)
Final output time series: df_output.shape = (273003, 3)
Data Preparation Summary:
Static Input Table: merged_df [2703 rows × 16 columns]
Time Series Output: df_output [273003 rows × 3 columns]
Cluster Boundaries: cluster_boundaries [(8, 4, 101)]


In [2]:
# Model Definition
class AdvancedDSSMDeepState(nn.Module):
    def __init__(self, input_dim, static_dim, hidden_dim, output_dim):
        super(AdvancedDSSMDeepState, self).__init__()

        # Static Data Path (Fully connected layers for static features)
        self.fc_static1 = nn.Linear(static_dim, 512)
        self.fc_static2 = nn.Linear(512, 256)
        self.fc_static3 = nn.Linear(256, 128)
        self.fc_static4 = nn.Linear(128, 64)

        # Time-series Path (Conv1D for feature extraction)
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=hidden_dim, kernel_size=3, padding=1)
        self.relu = nn.ReLU()

        # Deep State Dynamics (LSTM for latent state transitions)
        self.lstm_state = nn.LSTM(hidden_dim + 64, hidden_dim, batch_first=True)

        # Observation Model (Mapping latent states to outputs)
        self.fc1 = nn.Linear(hidden_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, time_series_input, static_input):
        # Static Data Path
        static_out = self.relu(self.fc_static1(static_input))
        static_out = self.relu(self.fc_static2(static_out))
        static_out = self.relu(self.fc_static3(static_out))
        static_out = self.relu(self.fc_static4(static_out))  # Shape: [batch_size, 64]

        # Time-Series Data Path
        if len(time_series_input.shape) == 2:  # [batch_size, seq_len]
            time_series_input = time_series_input.unsqueeze(1)  # Add channel dimension: [batch_size, 1, seq_len]

        conv_out = self.conv1(time_series_input)  # Conv1D layer
        conv_out = self.relu(conv_out)
        conv_out = conv_out.transpose(1, 2)  # Shape: [batch_size, seq_len, hidden_dim]

        # Expand static features to match the sequence length
        static_expanded = static_out.unsqueeze(1).expand(-1, conv_out.size(1), -1)  # Shape: [batch_size, seq_len, 64]

        # Combine Conv1D features and static features
        lstm_input = torch.cat([conv_out, static_expanded], dim=2)  # Shape: [batch_size, seq_len, hidden_dim + 64]

        # Latent State Dynamics (LSTM for state transitions)
        lstm_out, _ = self.lstm_state(lstm_input)  # Shape: [batch_size, seq_len, hidden_dim]

        # Observation Model
        lstm_out_final = lstm_out[:, -1, :]  # Use the last state for prediction
        x = self.fc1(lstm_out_final)
        x = self.relu(x)
        output = self.fc2(x)  # Final prediction

        return output


One important input at this point is the cluster_id of the test timeseries because the cluster data is being pulled using this and this is the line helping with that -

cluster_ids = np.array(Clustering_link_table['cluster'][Clustering_link_table['file_id'].isin(test_ids)])

test_ids is being reused from the last code block

Also the train-test split is hardcoded so it is a vulnerability.

In [3]:
def plot_discretized_validation(inputs, actuals, predictions, clusters, cluster_boundaries, model_name):
    file_id_to_index = {fid: idx for idx, fid in enumerate(Clustering_link_table['file_id'])}
    test_file_ids = list(Clustering_link_table['file_id'][Clustering_link_table['file_id'].isin(test_ids)])
    cluster_ids = np.array(Clustering_link_table['cluster'][Clustering_link_table['file_id'].isin(test_ids)])
    unique_clusters = np.unique(cluster_ids)

    y_length = predictions.shape[1]
    x_length = inputs.shape[1]
    num_timesteps = x_length + y_length

    # Track totals
    total_deviant_series = 0
    total_series = 0
    total_deviant_timesteps = 0
    total_timesteps = 0

    for cluster_id in unique_clusters:
        cluster_file_ids = [fid for i, fid in enumerate(test_file_ids) if cluster_ids[i] == cluster_id]
        test_pos_in_preds = [i for i, fid in enumerate(test_file_ids) if cluster_ids[i] == cluster_id]

        plt.figure(figsize=(7.5, 3.2))
        min_x = cluster_boundaries[cluster_id, 0, :x_length]
        max_x = cluster_boundaries[cluster_id, 3, :x_length]
        min_y = cluster_boundaries[cluster_id, 0, -y_length:]
        max_y = cluster_boundaries[cluster_id, 3, -y_length:]

        # Counters for current cluster
        deviant_series_count = 0
        deviant_timestep_count = 0

        for i, pred_idx in enumerate(test_pos_in_preds):
            pred = predictions[pred_idx]
            actual = actuals[pred_idx]

            # Plotting
            plt.plot(range(x_length), inputs[pred_idx], color='black', alpha=0.3, label='Input' if i == 0 else "")
            plt.plot(range(x_length, num_timesteps), actual, color='blue', alpha=0.5, label='Actual' if i == 0 else "")
            plt.plot(range(x_length, num_timesteps), pred, color='red', alpha=0.5, label='Predicted' if i == 0 else "")

            # Deviance checks
            deviant = False
            for t in range(y_length):
                if pred[t] < min_y[t] or pred[t] > max_y[t]:
                    deviant = True
                    deviant_timestep_count += 1

            if deviant:
                deviant_series_count += 1

        # Plot boundary shading
        plt.fill_between(range(x_length), min_x, max_x, color='orange', alpha=0.2, label='Input Boundary')
        plt.fill_between(range(x_length, num_timesteps), min_y, max_y, color='grey', alpha=0.3, label='Output Boundary')

        plt.xlabel('Time Steps')
        plt.ylabel('CO₂ Sequestration')
        plt.title(f'{model_name} – Cluster {cluster_id}')
        plt.legend()
        plt.tight_layout(pad=2.5)
        filename = f"drive/My Drive/DSSM-Figures2/{model_name}_cluster_{cluster_id}.pdf"
        plt.savefig(filename, format='pdf', bbox_inches='tight')
        plt.close()

        # Log percentages for current cluster
        cluster_total_series = len(test_pos_in_preds)
        cluster_total_timesteps = cluster_total_series * y_length
        pct_dev_series = 100 * deviant_series_count / cluster_total_series
        pct_dev_timesteps = 100 * deviant_timestep_count / cluster_total_timesteps
        print(f"Cluster {cluster_id} – Percent of deviant timeseries for {model_name}: {pct_dev_series:.2f}%")
        print(f"Cluster {cluster_id} – Percent of deviant timesteps for {model_name}: {pct_dev_timesteps:.2f}%")

        # Update global counts
        total_deviant_series += deviant_series_count
        total_deviant_timesteps += deviant_timestep_count
        total_series += cluster_total_series
        total_timesteps += cluster_total_timesteps

    # Print overall stats
    overall_series_pct = 100 * total_deviant_series / total_series
    overall_timestep_pct = 100 * total_deviant_timesteps / total_timesteps
    print(f'Overall – Percent of deviant timeseries: {overall_series_pct:.2f}%')
    print(f'Overall – Percent of deviant timesteps: {overall_timestep_pct:.2f}%')


In [4]:
def calculate_metrics(actuals, predictions,model_name):
    # Mean Absolute Error (MAE)
    mae = np.mean(np.abs(actuals - predictions), axis=1)
    mae_mean = np.mean(mae)
    mae_std = np.std(mae)

    # Mean Squared Error (MSE)
    mse = np.mean((actuals - predictions) ** 2, axis=1)
    mse_mean = np.mean(mse)
    mse_std = np.std(mse)

    # Symmetric Mean Absolute Percentage Error (SMAPE)
    smape = np.mean(2 * np.abs(actuals - predictions) / (np.abs(actuals) + np.abs(predictions) + 1e-8), axis=1) * 100
    smape_mean = np.mean(smape)
    smape_std = np.std(smape)

    # Root Mean Squared Error (RMSE)
    rmse = np.sqrt(mse)
    rmse_mean = np.mean(rmse)
    rmse_std = np.std(rmse)

    # R-squared (R²)
    ss_res = np.sum((actuals - predictions) ** 2, axis=1)
    ss_tot = np.sum((actuals - np.mean(actuals, axis=1, keepdims=True)) ** 2, axis=1)
    r2 = 1 - (ss_res / ss_tot)
    r2_mean = np.mean(r2)
    r2_std = np.std(r2)

    # Finding indices for the lowest, average, and highest RMSE
    min_rmse_index = np.argmin(rmse)
    max_rmse_index = np.argmax(rmse)
    avg_rmse_index = np.argsort(rmse)[len(rmse) // 2]  # median RMSE as the average case

    # Boundary case actuals and predictions
    Boundary_case_actuals = np.vstack([
        actuals[min_rmse_index],
        actuals[avg_rmse_index],
        actuals[max_rmse_index]
    ])

    Boundary_case_predicted = np.vstack([
        predictions[min_rmse_index],
        predictions[avg_rmse_index],
        predictions[max_rmse_index]
    ])

    # Return metrics and their standard deviations
    return {
        f'{model_name} MAE': mae_mean,
        f'{model_name} MAE_std': mae_std,
        f'{model_name} MSE': mse_mean,
        f'{model_name} MSE_std': mse_std,
        f'{model_name} SMAPE': smape_mean,
        f'{model_name} SMAPE_std': smape_std,
        f'{model_name} RMSE': rmse_mean,
        f'{model_name} RMSE_std': rmse_std,
        f'{model_name} R2': r2_mean,
        f'{model_name} R2_std': r2_std,
        f'Boundary_case_actuals': Boundary_case_actuals,
        f'Boundary_case_predicted': Boundary_case_predicted
    }

**Experiment to create the discretized validation table**

In [5]:
# --------------------- Deviation Table Init ---------------------
deviation_records = []

# --------------------- Training Loop ---------------------
splits = [(80, 20), (40, 60),(50, 50),(60, 40),(20, 80),(10, 90),(5, 95),(3, 97),(1, 99)]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
for train_pct, test_pct in splits:
    split_name = f"{train_pct}_{test_pct}"
    print(f"Running Split: {split_name}")

    # -- Prepare Train/Test IDs --
    train_ids, test_ids = train_test_split(df_output['file_id'].unique(), test_size=(test_pct/100), random_state=42)
    df_train = df_output[df_output['file_id'].isin(train_ids)]
    df_test = df_output[df_output['file_id'].isin(test_ids)]

    # -- Determine Timestep Lengths --
    train_timestep = int(train_pct / 100 * 101)

    # -- Construct Inputs and Outputs --
    pivot_train = df_train.pivot(index='file_id', columns='timestep', values='CO2')
    pivot_test = df_test.pivot(index='file_id', columns='timestep', values='CO2')

    X_train = pivot_train.values[:, :train_timestep]
    Y_train = pivot_train.values[:, train_timestep:]
    X_test = pivot_test.values[:, :train_timestep]
    Y_test = pivot_test.values[:, train_timestep:]

    static_train = merged_df[merged_df['file_id'].isin(train_ids)].drop(columns=['file_id', 'cluster']).values
    static_test = merged_df[merged_df['file_id'].isin(test_ids)].drop(columns=['file_id', 'cluster']).values

    # -- Tensor Conversion --
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    Y_train_tensor = torch.tensor(Y_train, dtype=torch.float32)
    static_train_tensor = torch.tensor(static_train, dtype=torch.float32)

    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    Y_test_tensor = torch.tensor(Y_test, dtype=torch.float32)
    static_test_tensor = torch.tensor(static_test, dtype=torch.float32)

    # -- DataLoader --
    train_dataset = TensorDataset(X_train_tensor, static_train_tensor, Y_train_tensor)
    test_dataset = TensorDataset(X_test_tensor, static_test_tensor, Y_test_tensor)
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=64)

    # -- Model Setup --
    model = AdvancedDSSMDeepState(input_dim=train_timestep,
                                  static_dim=static_train.shape[1],
                                  hidden_dim=101,
                                  output_dim=Y_train.shape[1])
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.MSELoss()

    # -- Training --
    model.to(device)
    model.train()
    for epoch in range(500):  # You can increase epochs here
        for Xb, Sb, Yb in train_loader:
            Xb, Sb, Yb = Xb.to(device), Sb.to(device), Yb.to(device)
            optimizer.zero_grad()
            preds = model(Xb, Sb)
            loss = criterion(preds, Yb)
            loss.backward()
            optimizer.step()
        print(f"{split_name} – Epoch {epoch + 1}, Loss: {loss.item():.5f}")

    # -- Evaluation --
    model.eval()
    predictions, actuals, inputs = [], [], []
    with torch.no_grad():
        for Xb, Sb, Yb in test_loader:
            Xb, Sb = Xb.to(device), Sb.to(device)
            outputs = model(Xb, Sb)
            predictions.append(outputs.cpu().numpy())
            actuals.append(Yb.numpy())
            inputs.append(Xb.cpu().numpy())

    predictions = np.concatenate(predictions, axis=0)
    actuals = np.concatenate(actuals, axis=0)
    inputs = np.concatenate(inputs, axis=0)

    model_name = f"DSSM_{split_name}"

    # ------------------ Deviation Calculation ------------------
    def calculate_discretized_deviation(preds, trues, test_ids, cluster_boundaries, clusters, x_length):
        y_length = preds.shape[1]
        cluster_ids = np.array(Clustering_link_table['cluster'][Clustering_link_table['file_id'].isin(test_ids)])
        unique_clusters = np.unique(cluster_ids)

        deviant_series_total = 0
        deviant_timesteps_total = 0
        total_series = 0
        total_timesteps = 0

        for cluster_id in unique_clusters:
            idxs = np.where(cluster_ids == cluster_id)[0]
            min_vals = cluster_boundaries[cluster_id, 0, -y_length:]
            max_vals = cluster_boundaries[cluster_id, 3, -y_length:]

            for i in idxs:
                deviant = False
                for t in range(y_length):
                    if preds[i][t] < min_vals[t] or preds[i][t] > max_vals[t]:
                        deviant_timesteps_total += 1
                        deviant = True
                if deviant:
                    deviant_series_total += 1

            total_series += len(idxs)
            total_timesteps += len(idxs) * y_length

        percent_series = 100 * deviant_series_total / total_series
        percent_timesteps = 100 * deviant_timesteps_total / total_timesteps
        return percent_series, percent_timesteps

    x_len = inputs.shape[1]
    percent_dev_series, percent_dev_timesteps = calculate_discretized_deviation(
        predictions, actuals, test_ids, cluster_boundaries, clusters, x_len)

    deviation_records.append({
        'Split': split_name,
        'Percent_Deviant_Timeseries': round(percent_dev_series, 2),
        'Percent_Deviant_Timesteps': round(percent_dev_timesteps, 2)
    })

    # -- Plotting --
    plot_discretized_validation(inputs, actuals, predictions, clusters, cluster_boundaries, model_name)

# --------------------- Save Final Deviation Table ---------------------
deviation_df = pd.DataFrame(deviation_records)
deviation_df.to_csv("drive/My Drive/DSSM-Figures/Deviation_Summary_Table.csv", index=False)
print("\n=== Deviation Summary Table ===")
print(deviation_df)

Using device: cuda
Running Split: 80_20
80_20 – Epoch 1, Loss: 0.19310
80_20 – Epoch 2, Loss: 0.16144
80_20 – Epoch 3, Loss: 0.14896
80_20 – Epoch 4, Loss: 0.10623
80_20 – Epoch 5, Loss: 0.13380
80_20 – Epoch 6, Loss: 0.11173
80_20 – Epoch 7, Loss: 0.04742
80_20 – Epoch 8, Loss: 0.03709
80_20 – Epoch 9, Loss: 0.02057
80_20 – Epoch 10, Loss: 0.00489
80_20 – Epoch 11, Loss: 0.00254
80_20 – Epoch 12, Loss: 0.00650
80_20 – Epoch 13, Loss: 0.00253
80_20 – Epoch 14, Loss: 0.00226
80_20 – Epoch 15, Loss: 0.00131
80_20 – Epoch 16, Loss: 0.00132
80_20 – Epoch 17, Loss: 0.00105
80_20 – Epoch 18, Loss: 0.00142
80_20 – Epoch 19, Loss: 0.00167
80_20 – Epoch 20, Loss: 0.00067
80_20 – Epoch 21, Loss: 0.00083
80_20 – Epoch 22, Loss: 0.00105
80_20 – Epoch 23, Loss: 0.00061
80_20 – Epoch 24, Loss: 0.00182
80_20 – Epoch 25, Loss: 0.00093
80_20 – Epoch 26, Loss: 0.00105
80_20 – Epoch 27, Loss: 0.00089
80_20 – Epoch 28, Loss: 0.00085
80_20 – Epoch 29, Loss: 0.00070
80_20 – Epoch 30, Loss: 0.00068
80_20 – E